In [1]:
import numpy as np
import pandas as pd

In [2]:
def find_split_indices(a):
    pairs = []
    i1 = 0
    label = a[i1]
    for i, x in enumerate(a[1:]):

        if label != a[i]:
            if label == 1:
                pair = [i1, i]
                pairs.append(pair)
            i1 = i
            label = a[i]

    if label == 1:
        pair = [i1, len(a)]
        pairs.append(pair)
    
    return pairs

In [3]:
def IoU(pred, true):
    
    intersection = (pred & true).sum()
    union = (pred | true).sum()

    if intersection * union == 0:
        iou = 0
    else:
        iou = (intersection / union).item()
        
    return iou

In [4]:
a = np.array([1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0])
b = np.array([0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0])

In [11]:
def calculate_metrics(a, b):
    
    pairs_a = find_split_indices(a)
    pairs_b = find_split_indices(b)
    
    na = len(pairs_a)
    nb = len(pairs_b)

    rows = []

    for pair_a in pairs_a:

        i1, i2 = pair_a

        match = True

        row = {
            "ref": False,
            "iou": None,
            "offset": None,
            "onset": None,
            "intersection": None,
            "union": None,
        }

        for pair_b in pairs_b:

            j1, j2 = pair_b

            if j2 < i1:
                continue

            elif j1 > i2:
                match = False
                break

            break

        if not match:
            rows.append(row)
            continue

        # row["ref"] = ib

        intersection = min(i2, j2) - max(i1, j1)
        union = max(i2, j2) - min(j1, i1)

        row["intersection"] = intersection
        row["union"] = union

        if intersection * union == 0:
            iou = 0
        else:
            iou = intersection / union      

        row["iou"] = iou

        onset = i1 - j1
        offset = i2 - j2

        row["onset"] = onset
        row["offset"] = offset

        rows.append(row)

    df = pd.DataFrame(rows)

    columns_metrics = ["iou", "offset", "onset"]

    metrics = {
        "n_diff": na - nb,
        
        "iou_mean": df["iou"].mean(),
        "iou_std": df["iou"].std(),

        "offset_mean": df["offset"].mean(),
        "offset_std": df["offset"].std(),
        
        "onset_mean": df["onset"].mean(),
        "onset_std": df["onset"].std()
    }
    
    return metrics

In [12]:
calculate_metrics(a, b)

{'n_diff': 1,
 'iou_mean': 0.55,
 'iou_std': 0.07071067811865474,
 'offset_mean': 0.5,
 'offset_std': 0.7071067811865476,
 'onset_mean': 0.0,
 'onset_std': 1.4142135623730951}